<a href="https://colab.research.google.com/github/akbardwi/Ripe-AI/blob/main/Machine%20Learning/Script/Training_Model_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kalau Mau Pakai Directory di Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Semua Dependencies

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD, RMSprop,Adam
from tensorflow.keras.models import model_from_json
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3

Set Directory Dataset

In [ ]:
BASE_DIR = '/content/drive/MyDrive/Dataset/Banana'      # Directory dataset 
TRAINING_DIR    = os.path.join(BASE_DIR, 'Training')
TESTING_DIR     = os.path.join(BASE_DIR, 'Test')
VALIDATION_DIR  = os.path.join(BASE_DIR, 'Validation')

Bikin Image Generator

In [ ]:
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=64
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=64
)

testing_generator = validation_datagen.flow_from_directory(
	TESTING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=64
)

Found 506 images belonging to 3 classes.
Found 146 images belonging to 3 classes.
Found 74 images belonging to 3 classes.


Import Pre-Trained Model

In [ ]:
inception_model = InceptionV3(input_shape = (150, 150, 3),    # ukuran input gambar
                                include_top = False,          # hilangkan layer dense
                                weights = 'imagenet')         # pakai data weight langsung dari imagenet

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
# inception_model.summary()

In [ ]:
last_layer = inception_model.get_layer('mixed5')           # coba eksperimen dengan layer yang berbeda
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


Bikin Layer untuk Output Model

In [ ]:
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)      # coba eksperimen dengan banyak layer yang berbeda
x = layers.Dropout(0.2)(x)                        # coba eksperimen dengan nilai dropout yang berbeda
x = layers.Dense(3, activation='softmax')(x)           

model = Model(inception_model.input, x) 

# model.summary()

Compile Model

In [ ]:
model.compile(optimizer = Adam(),               # coba eksperimen dengan optimizer yang berbeda
              loss = 'binary_crossentropy',     # coba eksperimen dengan loss function yang berbeda
              metrics = ['accuracy'])

Fit Data to Model

In [ ]:
history = model.fit(train_generator, epochs=25, steps_per_epoch=8, validation_data = validation_generator, verbose = 1, validation_steps=3)

Epoch 1/25
8/8 [==============================] - 263s 32s/step - loss: 5.8175 - accuracy: 0.5198 - val_loss: 23.6090 - val_accuracy: 0.5548
Epoch 2/25
8/8 [==============================] - 4s 515ms/step - loss: 0.3431 - accuracy: 0.8794 - val_loss: 184.5068 - val_accuracy: 0.5479
Epoch 3/25
8/8 [==============================] - 4s 512ms/step - loss: 0.0947 - accuracy: 0.9545 - val_loss: 19295.5703 - val_accuracy: 0.4247
Epoch 4/25
8/8 [==============================] - 4s 511ms/step - loss: 0.0943 - accuracy: 0.9605 - val_loss: 3462.1887 - val_accuracy: 0.4247
Epoch 5/25
8/8 [==============================] - 4s 516ms/step - loss: 0.0814 - accuracy: 0.9625 - val_loss: 58245.0977 - val_accuracy: 0.4247
Epoch 6/25
8/8 [==============================] - 4s 509ms/step - loss: 0.0731 - accuracy: 0.9625 - val_loss: 33118.0000 - val_accuracy: 0.4247
Epoch 7/25
8/8 [==============================] - 4s 516ms/step - loss: 0.0388 - accuracy: 0.9822 - val_loss: 14314.3945 - val_accuracy: 0.424

In [ ]:
skill = model.evaluate(testing_generator)

2/2 [==============================] - 7s 7s/step - loss: 0.0513 - accuracy: 0.9595


Saved Model ke TFLite

In [ ]:
import pathlib

# Export model
EXPORT_DIR = '/content/exportmodel'     # Directory export
tf.saved_model.save(model, EXPORT_DIR)

# Convert model
converter = tf.lite.TFLiteConverter.from_saved_model(EXPORT_DIR)
tflite_model = converter.convert()      

# Save model
tflite_model_file = pathlib.Path('/content/foo.tflite')
tflite_model_file.write_bytes(tflite_model)

Save Model ke TF Serving

In [ ]:
import tempfile

MODEL_DIR = '/content/model/Banana'        # Directory model. Apakah harus pakai temp directory?
version = 1
EXPORT_PATH = os.path.join(MODEL_DIR, str(version))   # Path version model
if os.path.isdir(EXPORT_PATH):
  print("\nPath already exist, cleaning up\n")
  !rm -r {EXPORT_PATH}

tf.keras.models.save_model(
    model,
    EXPORT_PATH,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print("\nSaved model:")
!ls -l {EXPORT_PATH}

INFO:tensorflow:Assets written to: /content/model/Banana/1/assets

Saved model:
total 3052
drwxr-xr-x 2 root root    4096 Jun  6 13:41 assets
-rw-r--r-- 1 root root  347018 Jun  6 13:41 keras_metadata.pb
-rw-r--r-- 1 root root 2768481 Jun  6 13:41 saved_model.pb
drwxr-xr-x 2 root root    4096 Jun  6 13:41 variables


In [ ]:
!zip -r /content/Banana.zip /content/model/Banana

  adding: content/model/Banana/ (stored 0%)
  adding: content/model/Banana/1/ (stored 0%)
  adding: content/model/Banana/1/saved_model.pb (deflated 92%)
  adding: content/model/Banana/1/assets/ (stored 0%)
  adding: content/model/Banana/1/variables/ (stored 0%)
  adding: content/model/Banana/1/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/model/Banana/1/variables/variables.index (deflated 79%)
  adding: content/model/Banana/1/keras_metadata.pb (deflated 95%)


In [ ]:
from google.colab import files
files.download('/content/Banana.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp /content/Banana.zip /content/drive/MyDrive/

In [ ]:
!saved_model_cli show --dir {EXPORT_PATH} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 150, 150, 3)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 3)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Concrete Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          input_1: TensorS